In [35]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression

In [36]:
#import iris data
from ucimlrepo import fetch_ucirepo 
  
# fetch dataset 
iris = fetch_ucirepo(id=53) 
  
# data (as pandas dataframes) 
X = iris.data.features 
y = iris.data.targets 
  
# metadata 
print(iris.metadata) 
  
# variable information 
print(iris.variables) 

{'uci_id': 53, 'name': 'Iris', 'repository_url': 'https://archive.ics.uci.edu/dataset/53/iris', 'data_url': 'https://archive.ics.uci.edu/static/public/53/data.csv', 'abstract': 'A small classic dataset from Fisher, 1936. One of the earliest known datasets used for evaluating classification methods.\n', 'area': 'Biology', 'tasks': ['Classification'], 'characteristics': ['Tabular'], 'num_instances': 150, 'num_features': 4, 'feature_types': ['Real'], 'demographics': [], 'target_col': ['class'], 'index_col': None, 'has_missing_values': 'no', 'missing_values_symbol': None, 'year_of_dataset_creation': 1936, 'last_updated': 'Tue Sep 12 2023', 'dataset_doi': '10.24432/C56C76', 'creators': ['R. A. Fisher'], 'intro_paper': {'title': 'The Iris data set: In search of the source of virginica', 'authors': 'A. Unwin, K. Kleinman', 'published_in': 'Significance, 2021', 'year': 2021, 'url': 'https://www.semanticscholar.org/paper/4599862ea877863669a6a8e63a3c707a787d5d7e', 'doi': '1740-9713.01589'}, 'add

In [37]:
#split dataset into shadow and target datasets

from sklearn.model_selection import train_test_split

# Splitting the dataset into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=42)

# "shadow" dataset
shadow_X, shadow_y = X_train, y_train

# "target" dataset
target_X, target_y = X_test, y_test

In [38]:
#create original target model
X_train, X_test, y_train, y_test = train_test_split(target_X, target_y, test_size=0.2, random_state=42)

target_original_model = LogisticRegression()
target_original_model.fit(X_train, y_train)

/Users/aadya/anaconda3/lib/python3.11/site-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LogisticRegression()

In [39]:
#create unlearned target model
index_unlearned = np.random.randint(0, len(target_X))

target_unlearned_X = target_X.drop(index=index_unlearned)
target_unlearned_y = target_y.drop(index=index_unlearned)

X_train_unlearned, X_test_unlearned, y_train_unlearned, y_test_unlearned = train_test_split(target_unlearned_X, target_unlearned_y, test_size=0.2, random_state=42)

target_unlearned_model = LogisticRegression()
target_unlearned_model.fit(X_train_unlearned, y_train_unlearned)

/Users/aadya/anaconda3/lib/python3.11/site-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LogisticRegression()

In [41]:
#split shadow dataset into positive and negative

#here,the code does a 80-20 split (positive recieves 80% and negative recieves 20%)
shadow_X_train, shadow_X_test, shadow_y_train, shadow_y_test = train_test_split(shadow_X, shadow_y, test_size=0.2, random_state=42)

#shadow "positive"
positive_shadow_X, positive_shadow_y = shadow_X_train, shadow_y_train

#shadow "negative"
negative_target_X, negative_target_y = shadow_X_test, shadow_y_test

In [42]:
#create original shadow model

X_train, X_test, y_train, y_test = train_test_split(positive_shadow_X, positive_shadow_y, test_size=0.2, random_state=42)

shadow_original_model = LogisticRegression()
shadow_original_model.fit(X_train, y_train)

/Users/aadya/anaconda3/lib/python3.11/site-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LogisticRegression()

In [44]:
#create unlearned shadow models

shadow_unlearned_models = {}

for i in range(len(positive_shadow_X)):
    temp_X = np.delete(positive_shadow_X, i, axis=0)
    temp_y = np.delete(positive_shadow_y, i, axis=0)
    
    model = LogisticRegression()
    
    model.fit(temp_X, temp_y)
    
    shadow_unlearned_models[f"shadow_unlearned_model_index_{i}"] = model

/Users/aadya/anaconda3/lib/python3.11/site-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/aadya/anaconda3/lib/python3.11/site-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/aadya/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/li

In [54]:
posteriors_list = [[], []]  

for i in range(len(positive_shadow_X)):
    unlearned_model = shadow_unlearned_models[f"shadow_unlearned_model_index_{i}"]

    original_model_prob = shadow_original_model.predict_proba([positive_shadow_X.iloc[i]])[0]
    unlearned_model_prob = unlearned_model.predict_proba([positive_shadow_X.iloc[i]])[0]
    
    posteriors_list[0].append(original_model_prob)
    posteriors_list[1].append(unlearned_model_prob)

posteriors_original_array = np.array(posteriors_list[0])
posteriors_unlearned_array = np.array(posteriors_list[1])

/Users/aadya/anaconda3/lib/python3.11/site-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(
/Users/aadya/anaconda3/lib/python3.11/site-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(
/Users/aadya/anaconda3/lib/python3.11/site-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(
/Users/aadya/anaconda3/lib/python3.11/site-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(
/Users/aadya/anaconda3/lib/python3.11/site-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(
/Users/aadya/anaconda3/lib/python3.

In [52]:
posteriors_array

array([[9.66055700e-01, 3.39415834e-02, 2.71669597e-06, 9.74955218e-01,
        2.50438897e-02, 8.92443364e-07],
       [7.76108477e-04, 2.52948175e-01, 7.46275716e-01, 3.92417946e-04,
        2.13261992e-01, 7.86345590e-01],
       [9.53844382e-01, 4.61473179e-02, 8.29960536e-06, 9.58437711e-01,
        4.15599218e-02, 2.36683219e-06],
       [1.74460604e-04, 1.33840572e-01, 8.65984967e-01, 1.08077699e-04,
        2.09307199e-01, 7.90584723e-01],
       [2.06782829e-02, 7.88976621e-01, 1.90345096e-01, 1.39927145e-02,
        8.51639639e-01, 1.34367647e-01],
       [9.51733689e-01, 4.82414519e-02, 2.48590865e-05, 9.42467763e-01,
        5.75270575e-02, 5.17959041e-06],
       [3.43238628e-04, 1.08704582e-01, 8.90952179e-01, 2.13077669e-04,
        1.50503635e-01, 8.49283288e-01],
       [9.34941249e-05, 4.73833363e-02, 9.52523170e-01, 5.04807003e-05,
        5.72711038e-02, 9.42678416e-01],
       [6.03536551e-02, 8.70612909e-01, 6.90334360e-02, 6.32986373e-02,
        8.26781971e-01, 